In [1]:
import numpy as np
import time
import cv2
import imutils
from imutils.video import FPS
from imutils.video import VideoStream
from matplotlib import pyplot as plt

import PIL.Image as Image
import os

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib
import csv


data_dir = pathlib.Path()
data_dir

#image data path set in dictionary
car_images_dict = {
    'suv': list(data_dir.glob('suv/*')),
    'sedan': list(data_dir.glob('sedan/*'))
}

#assigning 0 and 1 classification
car_labels_dict = {
    'suv': 0,
    'sedan': 1
}

X, y = [], []
#creating features and labels , X and y
for car_name, images in car_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(224,224))
        X.append(resized_img)
        y.append(car_labels_dict[car_name])
        
X = np.array(X)
y = np.array(y)       

#Spliting dataset for training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)        

#normalize image data
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

#pre-trained model imported from Tensorflow Hub
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

num_of_car_types = 2

model = tf.keras.Sequential([
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(num_of_car_types)#linear activation
])

model.summary()
model.compile(
  optimizer="adam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), #Computes the crossentropy loss between the labels 
    #and predictions
  metrics=['acc'])

model.fit(X_train_scaled, y_train, epochs=5)




WindowsPath('.')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________
Epoch 1/5
24/24 [==============================] - 23s 610ms/step - loss: 0.5884 - acc: 0.6942
Epoch 2/5
24/24 [==============================] - 13s 535ms/step - loss: 0.3601 - acc: 0.8711
Epoch 3/5
24/24 [==============================] - 13s 543ms/step - loss: 0.3219 - acc: 0.8433
Epoch 4/5
24/24 [==============================] - 13s 547ms/step - loss: 0.2557 - acc: 0.9119
Epoch 5/5
24/24 [==============================] - 13s 549ms/step - loss: 0.2371 - acc: 0.9115


In [2]:
#Creating a function for fetching per frame objects
def Objdect1(image):
    W,H = None,None
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),swapRB=True, crop=False)
    net.setInput(blob)
    if W is None or H is None:
        (H, W) = image.shape[:2]
    layerOutputs = net.forward(ln)
    
    
    
    # initialize our lists of detected bounding boxes, confidences, and
    # class IDs, respectively
    boxes = []
    confidences = []
    classIDs = []
    
    
    # loop over each layer output
    for output in layerOutputs:
        # loop over each detection
        for detect in output:
            # extracting class ID and confidence of the object detected
            scores = detect[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            
            
            # filtering out weaker predictions
            if confidence > CONFIDENCE_THRESHOLD:
                # rescale the bounding box coordinates to the size of image
                box = detect[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                
                # using center (x, y)-coordinates 
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                
                # updating bounding box coordinates
                boxes.append([x, y, int(width), int(height)])
                #updating confidences
                confidences.append(float(confidence))
                #updating class IDs
                classIDs.append(classID)
                
    # using non-maxima suppression to suppress weaker & overlapping bounding boxes
    nmxs = cv2.dnn.NMSBoxes(boxes, confidences, CONFIDENCE_THRESHOLD,CONFIDENCE_THRESHOLD)
    
    coordinates = []
    # at least 1 detection
    if len(nmxs) > 0:
        
        # loop over the indexes we are keeping
        for i in nmxs.flatten():
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            coordinates.append([x,y,x+w,y+h])
            
    for i in range(len(coordinates)):
        x1 = coordinates[i][0]
        y1 = coordinates[i][1]
        w1 = coordinates[i][2]
        h1 = coordinates[i][3]
        frame_new = image[y1:h1, x1:w1]
            
    return coordinates

    

In [4]:
start_time = time.time()

INPUT_FILE='assignmentclip.mp4'
OUTPUT_FILE='output.avi'
LABELS_FILE='cfg/coco.names'
CONFIG_FILE='cfg/yolov3.cfg'
WEIGHTS_FILE='yolov3.weights'
CONFIDENCE_THRESHOLD=0.3

import csv
H=None
W=None

fps = FPS().start()
vs = cv2.VideoCapture(INPUT_FILE)
fourcc = cv2.VideoWriter_fourcc(*"MJPG")

image_width = int(vs.get(3))
image_height = int(vs.get(4))
size = (image_width, image_height)
writer = cv2.VideoWriter(OUTPUT_FILE, fourcc, 30, size, True)

#LABELS = open(LABELS_FILE).read().strip().split("\n")

#np.random.seed(4)
#COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),dtype="uint8")


net = cv2.dnn.readNetFromDarknet(CONFIG_FILE, WEIGHTS_FILE)




# determine only the *output* layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
fcnt =0;

#writing csv file for car counts [Q1]
with open('cars_count.csv', 'w') as file1:
        csvwriter1 = csv.writer(file1,lineterminator='\n')
        csvwriter1.writerow(['Frame','CarCount'])
file1.close()

#writing csv file for car type counts [Q2]
with open('cartypes_count.csv', 'w') as file2:
        csvwriter2 = csv.writer(file2,lineterminator='\n')
        csvwriter2.writerow(['Frame','SUV','Sedan'])
file2.close()

#for each frames
while True and fcnt<150:
    fcnt+=1
    print ("Frame number", fcnt)
    try:
        (grabbed, image) = vs.read()
    except:
        break
    obj = Objdect1(image)
    #print(obj)
    with open('cars_count.csv', 'a') as file1:
        csvwriter1 = csv.writer(file1,lineterminator = '\n')
        csvwriter1.writerow([str(fcnt),len(obj)])
    file1.close()
    suv=0
    sed=0
    for i in range(len(obj)):
        x1 = obj[i][0]
        y1 = obj[i][1]
        w1 = obj[i][2]
        h1 = obj[i][3]
        try:
            frame_new = image[y1:h1, x1:w1]
            resizeimg = cv2.resize(frame_new, (224,224))
            resizeimg=np.array([resizeimg])
            n_car = model.predict(resizeimg/255)
            type01 = np.argmax(n_car)
            types=(list(car_labels_dict.keys())[list(car_labels_dict.values()).index(type01)])
            if type01==0:
                color=(0,255, 0)#green for suv
                suv+=1
            else:
                color=(127,0,63)#purple for sedan
                sed+=1
            cv2.rectangle(image, (x1, y1), (w1, h1),color,2)
            text = "{}".format(types)
            cv2.putText(image, text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX,0.5,color,2)
        except:
            pass
    
    
    
    with open('cartypes_count.csv', 'a') as file2:
        csvwriter2 = csv.writer(file2,lineterminator = '\n')
        csvwriter2.writerow([fcnt,suv,sed])
    file2.close()



    
    writer.write(image)
    fps.update()
    #plt.show()

fps.stop()

print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("--- %s seconds ---" % (time.time() - start_time))
#cleanup
cv2.destroyAllWindows()

# release the file pointers
print("cleaning up...")
writer.release()
vs.release()

15

16

Frame number 1


4

array([[[229, 199, 186],
        [229, 199, 186],
        [229, 199, 186],
        ...,
        [ 74,  85,  79],
        [ 68,  81,  75],
        [ 65,  78,  72]],

       [[229, 199, 186],
        [229, 199, 186],
        [229, 199, 186],
        ...,
        [ 67,  78,  72],
        [ 61,  74,  68],
        [ 58,  71,  65]],

       [[229, 199, 186],
        [229, 199, 186],
        [229, 199, 186],
        ...,
        [ 62,  73,  67],
        [ 55,  68,  62],
        [ 53,  66,  60]],

       ...,

       [[123, 138, 154],
        [123, 138, 154],
        [123, 138, 154],
        ...,
        [109, 118, 133],
        [109, 118, 133],
        [109, 118, 133]],

       [[123, 138, 154],
        [123, 138, 154],
        [123, 138, 154],
        ...,
        [109, 118, 133],
        [109, 118, 133],
        [109, 118, 133]],

       [[123, 138, 154],
        [123, 138, 154],
        [123, 138, 154],
        ...,
        [109, 118, 133],
        [109, 118, 133],
        [109, 118, 133]]

...

array([[[230, 201, 185],
        [230, 201, 185],
        [230, 201, 185],
        ...,
        [ 75,  88,  82],
        [ 63,  79,  72],
        [ 58,  74,  67]],

       [[230, 201, 185],
        [230, 201, 185],
        [230, 201, 185],
        ...,
        [ 72,  85,  79],
        [ 61,  77,  70],
        [ 55,  71,  64]],

       [[230, 201, 185],
        [230, 201, 185],
        [230, 201, 185],
        ...,
        [ 67,  80,  74],
        [ 55,  71,  64],
        [ 54,  70,  63]],

       ...,

       [[125, 138, 154],
        [125, 138, 154],
        [125, 138, 154],
        ...,
        [110, 119, 134],
        [110, 119, 134],
        [110, 119, 134]],

       [[126, 139, 155],
        [126, 139, 155],
        [126, 139, 155],
        ...,
        [110, 119, 134],
        [110, 119, 134],
        [110, 119, 134]],

       [[126, 139, 155],
        [126, 139, 155],
        [126, 139, 155],
        ...,
        [110, 119, 134],
        [110, 119, 134],
        [110, 119, 134]]

array([[[230, 201, 185],
        [230, 201, 185],
        [230, 201, 185],
        ...,
        [ 75,  88,  82],
        [ 63,  79,  72],
        [ 58,  74,  67]],

       [[230, 201, 185],
        [230, 201, 185],
        [230, 201, 185],
        ...,
        [ 72,  85,  79],
        [ 61,  77,  70],
        [ 55,  71,  64]],

       [[230, 201, 185],
        [230, 201, 185],
        [230, 201, 185],
        ...,
        [ 67,  80,  74],
        [ 55,  71,  64],
        [ 54,  70,  63]],

       ...,

       [[125, 138, 154],
        [125, 138, 154],
        [125, 138, 154],
        ...,
        [110, 119, 134],
        [110, 119, 134],
        [110, 119, 134]],

       [[126, 139, 155],
        [126, 139, 155],
        [126, 139, 155],
        ...,
        [110, 119, 134],
        [110, 119, 134],
        [110, 119, 134]],

       [[126, 139, 155],
        [126, 139, 155],
        [126, 139, 155],
        ...,
        [110, 119, 134],
        [110, 119, 134],
        [110, 119, 134]]

array([[[230, 201, 185],
        [230, 201, 185],
        [230, 201, 185],
        ...,
        [ 75,  88,  82],
        [ 63,  79,  72],
        [ 58,  74,  67]],

       [[230, 201, 185],
        [230, 201, 185],
        [230, 201, 185],
        ...,
        [ 72,  85,  79],
        [ 61,  77,  70],
        [ 55,  71,  64]],

       [[230, 201, 185],
        [230, 201, 185],
        [230, 201, 185],
        ...,
        [ 67,  80,  74],
        [ 55,  71,  64],
        [ 54,  70,  63]],

       ...,

       [[125, 138, 154],
        [125, 138, 154],
        [125, 138, 154],
        ...,
        [110, 119, 134],
        [110, 119, 134],
        [110, 119, 134]],

       [[126, 139, 155],
        [126, 139, 155],
        [126, 139, 155],
        ...,
        [110, 119, 134],
        [110, 119, 134],
        [110, 119, 134]],

       [[126, 139, 155],
        [126, 139, 155],
        [126, 139, 155],
        ...,
        [110, 119, 134],
        [110, 119, 134],
        [110, 119, 134]]

8

[INFO] elasped time: 184.65
--- 184.6543323993683 seconds ---
cleaning up...
